In [ ]:
import sys
print(sys.executable)

# python 3.7 is needed for azure cognitive speech services

In [8]:
import azure.cognitiveservices.speech as speechsdk
from keys import *


1. constant voice recognition
2. intermittent translation
3. speech model -> transcribe -> tts

4. test with video of dysarthric speaker.

In [12]:
def speech_recognize_continuous_async_from_microphone():
    """performs continuous speech recognition asynchronously with input from microphone"""
    speech_config = speechsdk.SpeechConfig(subscription=subscription, region="westus")
    # The default language is "en-us".
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

    done = False

    def recognizing_cb(evt: speechsdk.SpeechRecognitionEventArgs):
    
        print('RECOGNIZING: {}'.format(evt))

    def recognized_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        
        print(evt.result.text)
        #print('RECOGNIZED: {}'.format(evt))

    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    # Connect callbacks to the events fired by the speech recognizer
    #speech_recognizer.recognizing.connect(recognizing_cb)
    speech_recognizer.recognized.connect(recognized_cb)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Perform recognition. `start_continuous_recognition_async asynchronously initiates continuous recognition operation,
    # Other tasks can be performed on this thread while recognition starts...
    # wait on result_future.get() to know when initialization is done.
    # Call stop_continuous_recognition_async() to stop recognition.
    result_future = speech_recognizer.start_continuous_recognition_async()

    result_future.get()  # wait for voidfuture, so we know engine initialization is done.
    print('Continuous Recognition is now running, say something.')

    while not done:
        # No real sample parallel work to do on this thread, so just wait for user to type stop.
        # Can't exit function or speech_recognizer will go out of scope and be destroyed while running.
        print('type "start" to begin')
        stop = input()
        if (stop.lower() == "start"):
            print('Starting async recognition.')
            speech_recognizer.stop_continuous_recognition_async()
            done = True
            break

    print("Starting async continuous recognition.")
    #speech_recognizer.stop_continuous_recognition_async()




In [13]:
speech_recognize_continuous_async_from_microphone()


Continuous Recognition is now running, say something.
type "start" to begin
type "start" to begin
Check. Check.
type "start" to begin
Hello. Hello. Hello.
type "start" to begin
Hello. Hello. Hello.
type "start" to begin
type "start" to begin
Testing. Testing.
type "start" to begin


KeyboardInterrupt: Interrupted by user

OK.



Stop.

A lot of sugar, man, no.

It's mercury. Mercury poisoning.
Yes, and too much is more than like twice a month or some ****.
There's a lot of mercury into my mouth.
Check the check the ******* numbers. I don't ******* remember.
It's it's a decent amount. It's not very healthy.
I mean, I don't know, but look it up.
Antennas.


In [ ]:
def from_mic():

        speech_config = speechsdk.SpeechConfig(subscription= subscription, region="westus")
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)
        print("Speak into your microphone.")
        result = speech_recognizer.recognize_once_async().get()
        print(type(result))
        print(result.text)

from_mic()


In [ ]:
def stt():

    config = speechsdk.SpeechConfig(subscription=subscription, region="westus")
    config.endpoint_id = endpoint

    def from_file():
        audio = speechsdk.AudioConfig(filename="data/my_tip_number_one.wav")
        speech_recognizer = speechsdk.SpeechRecognizer(speech_config=config, audio_config=audio)

        result = speech_recognizer.recognize_once_async().get()
        print(result.text)

    from_file()

In [ ]:
# REST API
# restLink = "https://westus3.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1?cid=398d0cf9-b231-4521-912f-f208610e48f1"

def tss():
    speech_config = speechsdk.SpeechConfig(subscription= subscription, region='westus')
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

    # The language of the voice that speaks.
    speech_config.speech_synthesis_voice_name='en-US-JennyNeural'

    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

    # Get text from the console and synthesize to the default speaker.
    print("Enter some text that you want to speak >")
    text = "hello my name is"

    speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()

    if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized for text [{}]".format(text))
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print("Speech synthesis canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(cancellation_details.error_details))
                print("Did you set the speech resource key and region values?")

In [ ]:
stt()

In [ ]:
from_mic()

In [ ]:
tss()